In [4]:
#pip install opencv-python
#pip install chrono
#pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import face_recognition
import cv2

wkngdir = os.getcwd()
folder = wkngdir+"/Results/uniquepersons"

known_faces = []
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        lmm_image = face_recognition.load_image_file(file_path)
        lmm_face_encoding = face_recognition.face_encodings(lmm_image)[0]
        known_faces.append(lmm_face_encoding)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

        
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0

while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)

        # If you had more than 2 faces, you could make this logic a lot prettier
        # but I kept it simple for the demo
        name = None
        if match[0]:
            name = "Lin-Manuel Miranda"
        elif match[1]:
            name = "Alex Lacamoire"

        face_names.append(name)

    # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    print("Writing frame {} / {}".format(frame_number, length))
    output_movie.write(frame)

# All done!
input_movie.release()
cv2.destroyAllWindows()

In [35]:
os.chdir(folder)
video_out_dir = "videoByPerson"
os.makedirs(video_out_dir,exist_ok=True)
os.chdir(wkngdir)
video_out_dir = "Results2//"+video_out_dir

video_capture = cv2.VideoCapture(0)
data = pd.DataFrame(columns=['person','framecount'])

count=0
video_array = []

In [36]:
def checktheframe(face_encodings,persons_dir):
    matched = False
    iter_t = 0
    for filename in os.listdir(persons_dir):
        path = os.path.join(persons_dir,filename)
        filter_image = face_recognition.load_image_file(path)
        filter_face_encoding = face_recognition.face_encodings(filter_image)
        known_face_encodings.append(filter_face_encoding[0])
        known_face_names.append(filename)
        matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
        if True in matches:
            matched = True
            data['framecount'] += 1
            video_array[iter_t].write(frame)
            break;
        iter_t = iter_t+1;       
    return matched

In [37]:
while True:
    ret, frame = video_capture.read()
    process_this_frame = True
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        video_capture.release();
        print("Escape hit, closing...") 
        break
    if not (ret):
        logging.error("Could not read frame from camera. Stopping video capture.")
        break
   
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    height, width, layers = frame.shape
    size = (width,height)
    if process_this_frame:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        if len(face_encodings) > 0:
            matched = checktheframe(face_encodings,persons_dir)
            if matched == False:
                cv2.imwrite(os.path.join(persons_dir,"person_"+str(count)+".jpg"),frame)
                data = data.append({'person':os.path.join(persons_dir,"person_"+str(count)),'framecount':1},ignore_index=True)  
                temp_video = cv2.VideoWriter(os.path.join(video_out_dir,"videoex_"+str(count)+".avi"),
                                             cv2.VideoWriter_fourcc(*'DIVX'),15,size,isColor = True)
                video_array.append(temp_video)
                count+=1
            
          
cv2.destroyAllWindows();
data.to_excel("VideoAnalysis1.xlsx")
for t in video_array:
    t.release()
            
print(data)
print(video_array)


Escape hit, closing...
                            person framecount
0  Results//uniquepersons\person_0          5
[<VideoWriter 000002558A481270>]


In [26]:
video_capture.release();
cv2.destroyAllWindows();